In [2]:
### Requirements
# pip install lightfm

# Imports & Preparation

In [1]:
# Public
import pandas as pd
import numpy as np

from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.evaluation import auc_score

# Custom
from data_preparation import get_data
from lightfmFunctions import get_recoms
from support_functions import get_top_sorted

from classes import Mappings

In [3]:
data, fm_data, users, user_ids, items, item_ids = get_data()

# Model

In [4]:
dataset = Dataset()
dataset.fit(users=user_ids, items=item_ids)

mappings = Mappings(dataset)

interactions, _ = dataset.build_interactions(fm_data.to_records(index=False))
model = LightFM(loss="warp", learning_rate=0.05, random_state=42)
model.fit(interactions=interactions, epochs=100)

In [5]:
# get_recoms(fm_data=fm_data, items=items, dataset=dataset, model=model, mappings=mappings, user_id=2, num_recs=10)

In [7]:
recs_for_all_users = pd.DataFrame()
for i in user_ids:
    print(i)
    recs_for_all_users = recs_for_all_users.append(
        get_recoms(fm_data=fm_data, items=items, dataset=dataset, model=model, mappings=mappings, user_id=i, num_recs=10)
    )

24
23
2
12
3
31
32
27
34
35
21
20
9
36
17
14
37
30
13
18
16
28
48
51
53
59
58
22
61
49
63
64
76
33
46
87
94
96
91
97
100
103
108
110
113
138
144
159
164
165
171
173
175
180
190


ValueError: The user feature matrix specifies more features than there are estimated feature embeddings: 183 vs 191.